In [2]:
import numpy as np
import numpy.linalg as la
from random import randrange
import matplotlib.pyplot as pt

nnodes = 10
circuit = [{i+1: randrange(1, 50) * 100}
            if i+1 < nnodes
            else {}
           for i in range(nnodes)]

nedges = int(nnodes**2 * 0.2)

for i in range(nedges):
    ijnode = randrange((nnodes-1)*nnodes/2)
    jnode = 0
    ijnode0 = 0
    while True:
        jnode += 1
        ijnode0 += jnode
        if ijnode0 > ijnode:
            break
    inode = ijnode - ijnode0 + jnode
    circuit[inode][jnode] = randrange(1, 50) * 100

fixed_voltages = {0: 5, nnodes-1: 0}

def net_to_dot(net, fixed_voltages, voltages=None):
    lines = []
    for from_node_nr in range(len(net)):
        connections = net[from_node_nr]
        for to_node_nr, item in connections.items():
            label = "%r Ohms" % item
            arrow = "none"
            color = "black"

            lines.append(
            'node%d -> node%d [label="%s",arrowhead="%s", fontcolor="%s"];'
            % (from_node_nr, to_node_nr, label, arrow, color))

    if voltages is not None:
        for node_nr in range(len(net)):
            lines.append(
            'node%d [label="node%d: %.2f V"];'
            % (node_nr, node_nr, voltages[node_nr]))

    for node_nr, voltage in fixed_voltages.items():
        lines.append('node%d -> voltage%d [arrowhead="none"];\n'
                 'voltage%d [label="%s",shape=box];\n'
                 % (node_nr, node_nr, node_nr, "%r V" % voltage))

    return "digraph network {\n%s\n}" % "\n".join(lines)

def plot_circuit(circuit, fixed_voltages, voltages=None):
    from tempfile import TemporaryDirectory
    from os.path import join
    from subprocess import check_call

    with TemporaryDirectory() as td:
        dot_name = "x.dot"
        with open(join(td, dot_name), "w") as dotf:
            dotf.write(net_to_dot(circuit, fixed_voltages, voltages=voltages))

        png_name = "x.png"
        with open(join(td, png_name), "wb") as pngf:
            check_call(["dot", "-Gdpi=200", dot_name, "-Tpng"], stdout=pngf, cwd=td)

        #from scipy.misc import imread
        #png_data = imread(join(td, png_name))
        from PIL import Image
        im       = Image.open(join(td, png_name))
        a        = np.asarray(im)
        png_data = Image.fromarray(a)

        pt.figure(figsize=(13,10))
        pt.imshow(png_data)
        pt.setp(pt.gca().get_xticklabels(), visible=False)
        pt.setp(pt.gca().get_yticklabels(), visible=False)
        pt.setp(pt.gca().get_xticklines(), visible=False)
        pt.setp(pt.gca().get_yticklines(), visible=False)

/tmp/ipykernel_22336/2882480949.py:15: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  ijnode = randrange((nnodes-1)*nnodes/2)


In [26]:
L = np.zeros(shape=(10,10))
for i in range(10):
    for j in range(10):
        if j in circuit[i]:
            L[i][j] = -1/circuit[i][j]
        elif i in circuit[j]:
            L[i][j] = -1/circuit[j][i]
        if i==j:
            cur=0
            for k in range(10):
                if k in circuit[i]:
                    cur += 1/circuit[i][k]
                elif i in circuit[k]:
                    cur += 1/circuit[k][i]
            L[i][j]=cur
L[0][0] = 0 
L[9][1:9] = 0
print(L[0])
print(L[9])
u=np.zeros(10)
u[0] = -fixed_voltages[0]
la.solve(L,u)

[ 0.         -0.00022222  0.          0.         -0.002       0.
 -0.0005      0.         -0.00037037 -0.00058824]
[-0.00058824  0.          0.          0.          0.          0.
  0.          0.          0.          0.01155988]


array([2701.40648238, 1641.08961013, 1041.53944236, 1309.26752612,
       1873.91831619,  848.11109647, 1338.26310852,  818.63278513,
        371.2075093 ,  137.46365007])